###Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies.head(2)

In [ ]:
movies.shape

In [ ]:
credits.head(2)

In [ ]:
movies = movies.merge(credits, on="title")

In [ ]:
movies.head(2)

In [ ]:
movies.info()

###Taking column for recomendation
* 'movie_id',
* 'title',
* 'overview',
* 'genres','keywords',
* 'cast',
* 'crew'

In [ ]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

###Create a helper function to convert unorganised dict to a organised list

In [ ]:
def converter(gen):
    L = []
    for genres in ast.literal_eval(gen):
        L.append(genres['name'])
    return L

In [ ]:
movies["genres"] = movies['genres'].apply(converter)

In [ ]:
movies.head(2)movies['keywords'][0]

In [ ]:
movies['keywords'] = movies['keywords'].apply(converter)

In [ ]:
movies.head(2)

###Take first 3 name from cast column

In [ ]:
def converter2(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter != 3:
            L.append(i['name'])
            counter += 1
        else:
            break
    return L

In [ ]:
movies['cast'][0]

In [ ]:
movies['cast'] = movies['cast'].apply(converter2)

In [ ]:
movies.head(2)

In [ ]:
movies['cast'][0]

In [ ]:
movies['crew'][0]

In [ ]:
def find_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == "Director":
            L.append(i['name'])
    return L

In [ ]:
movies['crew'] = movies['crew'].apply(find_director)

In [ ]:
movies.head(2)

###Remove extra space from list word

In [ ]:
def remove_space(text):
    L1 = []
    for i in text:
        L1.append(i.replace(" ", ""))
    return L1


In [ ]:
movies['cast'] = movies['cast'].apply(remove_space)
movies['genres'] = movies['genres'].apply(remove_space)
movies['keywords'] = movies['keywords'].apply(remove_space)
movies['crew'] = movies['crew'].apply(remove_space)

In [ ]:
movies.head(2)

In [ ]:
movies['overview'] = movies['overview'].fillna('')  # Replace NaN with empty string

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split()) #Spliting overview to single word

In [ ]:
# merge all column except movie_id and title and put it to tags column
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
movies.head(2)

In [ ]:
#remove merge column
movies = movies.drop(columns=['overview','genres','keywords','cast','crew'])

In [ ]:
movies.head(2)

In [ ]:
# In tags column remove all comma joint all together to create a paragraph
movies["tags"] = movies['tags'].apply(lambda x : " ".join(x))


In [ ]:
movies.head(2)

In [ ]:
movies["tags"] = movies['tags'].apply(lambda x : x.lower()) # take all latter in lower case

In [ ]:
movies.head(2)

###remove same type worb from list

In [ ]:
remove same type worb from list

In [ ]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
movies['tags'] = movies['tags'].apply(stem)

In [ ]:
movies.head(2)

In [ ]:
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vector = cv.fit_transform(movies['tags']).toarray()

In [ ]:
vector.shape

In [ ]:
cv.get_feature_names_out()

In [ ]:
features = cv.get_feature_names_out()
print(list(features))

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
similarity[0]

In [ ]:
movies[movies['title'] == 'Avatar'].index[0]

In [ ]:
#create a function for recomendation
def recommendation(movie):
    movie_index = movies[movies['title'] == movie].index[0]
    movie_list = sorted(list(enumerate(similarity[movie_index])), reverse=True, key = lambda x : x[1])[1:6]
    for i in movie_list:
        print(movies.iloc[i[0]].title)



In [ ]:
recommendation("Avatar")

In [ ]:
pickle.dump(movies.to_dict(), open("movie_dict.pkl", 'wb'))


In [ ]:
pickle.dump(similarity, open("similarity.pkl", "wb"))